# 自分で実装
ソルバを使わずに自力で実装．茨の道．

In [2]:
using Plots
using LinearAlgebra

In [3]:
# テスト用のシステムを定義
x₀ = [1, 0.5]
test = (
    A = [
        1.1 2
        -0.3 -1
    ],
    B = [
        1 2
        0.847 3
    ],
    C = [
        1. 0.
        0. 1.
    ],
    Q = diagm([10.0, 10.0]),
    R = diagm([1., 1.])
);

ソルバを使って求めたリカッティ方程式の解

In [4]:
using MatrixEquations
P_solver, _, _ = arec(test.A, test.B, test.R, test.Q, zero(test.B))
P_solver

2×2 Matrix{Float64}:
  3.34568  -1.07266
 -1.07266   1.30235

# 微分方程式を解いて求める
力技．こちらは簡単．  

In [5]:
"""離散時間リカッティ方程式を解く"""
function solve_dare(A, B, Q, R, max=100000, step=0.01)
    P = zero(Q)  # 初期値
    P_old = zero(Q)

    for i in 1:max
        copy!(P_old, P)
        P = P .+ (P*A .+ A'*P .- P*B*inv(R)*B'*P .+ Q) .* step
        if abs.(P_old .- P) |> maximum < 1e-6
            break
        end
    end

    P
end

P = solve_dare(test.A, test.B, test.Q, test.R)
P

2×2 Matrix{Float64}:
  3.34567  -1.07265
 -1.07265   1.30235

ソルバを使って求めた解と比較

In [6]:
P - P_solver

2×2 Matrix{Float64}:
 -1.64684e-5   7.15116e-6
  7.15116e-6  -3.10528e-6

ほぼ合ってる．  

****
## 有本・ポッターの方法


In [1]:
"""有本・ポッター"""
function aimoto_poter(A, B, Q, R)
    
    ℋ = [
        A' -B*inv(R)*B'
        -Q -A
    ]  #  ハミルトン行列
    #println("ℋ = ", ℋ)

    λ_ = eigvals(ℋ)  # ハミルトン行列の固有値
    ω_ = eigvecs(ℋ)  # ハミルトン行列の固有ベクトル
    println("ℋ の固有値 = ", λ_)
    println("ℋ の固有ベクトル", ω_)

    # ハミルトン行列の固有値が負のものを探す
    index = []
    for i in 1:size(λ_)[1]
        print(i)
        if real(λ_[i]) < 0
            println("実部が負")
            push!(index, i)
        else
            println("実部が正")
        end
    end
    println("ℋ の固有値の実部が負なのは, ", index)
    n = size(index)[1]

    # Y,Zを計算
    ω = Matrix{Float64}(undef, size(ω_)[1], n)  # 未初期化のMatrix

    for i in 1:n
        ω[:, i] .= ω_[:, index[i]]
    end

    Y = ω[1:n, :]
    Z = ω[n+1:end, :]

    P = Z' * inv(Y')
    return P

end

P_arimoto = aimoto_poter(test.A, test.B, test.Q, test.R)
P_arimoto

UndefVarError: UndefVarError: test not defined